#### Kernel using 4 neighbors

In [1]:
import os
import numpy as np
import cv2
from scipy.optimize import minimize
from tqdm import tqdm

# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.random.rand(3, 3)

# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    return 1 / (1 + (s_squared + lambda_value**2))

# Objective function to minimize
def objective_function(kernel_params, clean_images, noisy_images):
    kernel = kernel_params.reshape((3, 3))
    kernel_padded = np.pad(kernel, padding, 'constant')
    error = 0
    for clean_image, noisy_image in zip(clean_images, noisy_images):
        denoised_image = noisy_image.copy()
        for _ in range(20):
            denoised_image = denoise_image(denoised_image, kernel_padded)
        error += np.mean((denoised_image - clean_image) ** 2)  # Example error metric (MSE)
    return error

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u * g(k_u**2)
    p = dt * phi 
    return image + p

# Load the training images (clean and noisy) from separate directories
clean_dir = 'clean_images'
noisy_dir = 'mult_train'

clean_images = []
noisy_images = []

for filename in os.listdir(clean_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        clean_image = cv2.imread(os.path.join(clean_dir, filename), 0)
        clean_image = clean_image.astype(np.float32) / 255.0
        clean_images.append(clean_image)

        noisy_image = cv2.imread(os.path.join(noisy_dir, filename), 0)
        noisy_image = noisy_image.astype(np.float32) / 255.0
        noisy_images.append(noisy_image)

# Flatten initial kernel for optimization
initial_params = initial_kernel.flatten()

# Optimize the kernel parameters with tqdm progress bar
with tqdm(total=len(clean_images), desc='Training Kernel') as pbar:
    def update_progress(_):
        pbar.update(1)

    # Optimize the kernel parameters
    optimized_params = minimize(objective_function, initial_params, args=(clean_images, noisy_images),
                                callback=update_progress)

# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image = cv2.imread('test.png', 0)
test_image = test_image.astype(np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = test_image.copy()
for _ in range(200):
    denoised_image = denoise_image(denoised_image, trained_kernel_padded)

cv2.imwrite('perona2.jpg', denoised_image * 255.0)

Training Kernel:   0%|▏                                                              | 1/400 [00:16<1:48:31, 16.32s/it]


True

In [2]:
print(trained_kernel)

[[0.28771478 0.45686923 0.59686541]
 [0.69384825 0.41176739 1.61321642]
 [0.97749501 0.98236024 0.48730316]]


In [3]:
import os
import numpy as np
import cv2
from scipy.optimize import minimize
from tqdm import tqdm

# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.random.rand(3, 3)

# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# ReLU activation function
def relu(x):
    return np.maximum(0, x)

# Function to compute phi(x) using ReLU activation
def g(x):
    return relu(x)

# Objective function to minimize
def objective_function(kernel_params, clean_images, noisy_images):
    kernel = kernel_params.reshape((3, 3))
    kernel_padded = np.pad(kernel, padding, 'constant')
    error = 0
    for clean_image, noisy_image in zip(clean_images, noisy_images):
        denoised_image = noisy_image.copy()
        for _ in range(20):
            denoised_image = denoise_image(denoised_image, kernel_padded)
        error += np.mean((denoised_image - clean_image) ** 2)  # Example error metric (MSE)
    return error

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    p = dt * k_u * g(k_u)
    return image + p

# Load the training images (clean and noisy) from separate directories
clean_dir = 'clean_images'
noisy_dir = 'mult_train'

clean_images = []
noisy_images = []

for filename in os.listdir(clean_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        clean_image = cv2.imread(os.path.join(clean_dir, filename), 0)
        clean_image = clean_image.astype(np.float32) / 255.0
        clean_images.append(clean_image)

        noisy_image = cv2.imread(os.path.join(noisy_dir, filename), 0)
        noisy_image = noisy_image.astype(np.float32) / 255.0
        noisy_images.append(noisy_image)

# Flatten initial kernel for optimization
initial_params = initial_kernel.flatten()

# Optimize the kernel parameters with tqdm progress bar
with tqdm(total=len(clean_images), desc='Training Kernel') as pbar:
    def update_progress(_):
        pbar.update(1)

    # Optimize the kernel parameters
    optimized_params = minimize(objective_function, initial_params, args=(clean_images, noisy_images),
                                callback=update_progress)

# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image = cv2.imread('test.png', 0)
test_image = test_image.astype(np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = test_image.copy()
for _ in range(200):
    denoised_image = denoise_image(denoised_image, trained_kernel_padded)

cv2.imwrite('relu2.jpg', denoised_image * 255.0)

Training Kernel:   0%|                                                                         | 0/400 [00:00<?, ?it/s]C:\Users\anish\AppData\Local\Temp\ipykernel_10996\532754169.py:41: RuntimeWarning: overflow encountered in multiply
  p = dt * k_u * g(k_u)
C:\Users\anish\AppData\Local\Temp\ipykernel_10996\532754169.py:35: RuntimeWarning: overflow encountered in square
  error += np.mean((denoised_image - clean_image) ** 2)  # Example error metric (MSE)
C:\Users\anish\AppData\Roaming\Python\Python39\site-packages\numpy\core\_methods.py:180: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
Training Kernel:   0%|                                                                         | 0/400 [00:07<?, ?it/s]


True

In [4]:
print(trained_kernel)

[[0.04449146 0.13575232 0.24767822]
 [0.33346809 0.4981568  0.55886843]
 [0.91934352 0.09844713 0.55023631]]


In [5]:
import os
import numpy as np
import cv2
from scipy.optimize import minimize
from tqdm import tqdm

# Define the parameters
dt = 0.1  # Time step
initial_kernel = np.random.rand(3, 3)

# Pad the kernel to match the size of the image
padding = (initial_kernel.shape[0] - 1) // 2
initial_kernel_padded = np.pad(initial_kernel, padding, 'constant')

# Define the hyperparameter lambda
lambda_value = 0.1

# Function to compute g(s^2)
def g(s_squared):
    denom = np.sqrt(1 + s_squared/lambda_value**2)
    return np.sqrt(s_squared)/denom

# Objective function to minimize
def objective_function(kernel_params, clean_images, noisy_images):
    kernel = kernel_params.reshape((3, 3))
    kernel_padded = np.pad(kernel, padding, 'constant')
    error = 0
    for clean_image, noisy_image in zip(clean_images, noisy_images):
        denoised_image = noisy_image.copy()
        for _ in range(20):
            denoised_image = denoise_image(denoised_image, kernel_padded)
        error += np.mean((denoised_image - clean_image) ** 2)  # Example error metric (MSE)
    return error

# Function to apply the update rule
def denoise_image(image, kernel):
    k_u = cv2.filter2D(image, -1, kernel)
    phi = k_u*g(k_u**2)
    p = dt * phi
    return image + p

# Load the training images (clean and noisy) from separate directories
clean_dir = 'clean_images'
noisy_dir = 'mult_train'

clean_images = []
noisy_images = []

for filename in os.listdir(clean_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        clean_image = cv2.imread(os.path.join(clean_dir, filename), 0)
        clean_image = clean_image.astype(np.float32) / 255.0
        clean_images.append(clean_image)

        noisy_image = cv2.imread(os.path.join(noisy_dir, filename), 0)
        noisy_image = noisy_image.astype(np.float32) / 255.0
        noisy_images.append(noisy_image)

# Flatten initial kernel for optimization
initial_params = initial_kernel.flatten()

# Optimize the kernel parameters with tqdm progress bar
with tqdm(total=len(clean_images), desc='Training Kernel') as pbar:
    def update_progress(_):
        pbar.update(1)

    # Optimize the kernel parameters
    optimized_params = minimize(objective_function, initial_params, args=(clean_images, noisy_images),
                                callback=update_progress)

# Reshape optimized parameters to obtain the trained kernel
trained_kernel = optimized_params.x.reshape((3, 3))

# Pad the trained kernel
trained_kernel_padded = np.pad(trained_kernel, padding, 'constant')

# Load the test image
test_image = cv2.imread('test.png', 0)
test_image = test_image.astype(np.float32) / 255.0

# Denoise the test image using the trained kernel
denoised_image = test_image.copy()
for _ in range(200):
    denoised_image = denoise_image(denoised_image, trained_kernel_padded)

cv2.imwrite('charbonier2.jpg', denoised_image * 255.0)

Training Kernel:   1%|▊                                                              | 5/400 [06:19<8:19:03, 75.81s/it]


True

In [6]:
print(trained_kernel)

[[ 0.5396388  -0.57564548  0.97759542]
 [ 0.28766341 -2.24797799 -1.88638612]
 [ 0.33712107  0.78464178  2.43901603]]
